In [0]:
# word2vec implementation to compute vector implementation of words

from __future__ import print_function, division, absolute_import

import collections
import os
import random
import urllib
import zipfile

import numpy as np
import tensorflow as tf


In [0]:
# Training hyper-parameters
learning_rate = 0.1
batch_size = 128
num_steps = 1000000
display_steps = 10000
eval_steps = 200000

In [0]:
# evaluation parameters
eval_words = ['five', 'of', 'going', 'hardware', 'american', 'britain']

In [0]:
# word2vec parameters
embedding_size = 200 # dimension of embedding vector
max_vocabulary_size = 50000 # total number of different words in vocabulary
min_occurence = 10 # remove all words that does not appear at least n times
skip_window = 3 # how many words to consider left and right
num_skips = 2 # how many times to reuse an input to generate a label
num_sampled = 64 # number of negative examples to sample


In [0]:
# download small chunk of wikipedia text collection
url = 'http://mattmahoney.net/dc/text8.zip'

data_path = 'text8.zip'

if not os.path.exists(data_path):
  print("Downloading dataset ... (It may take some time)")
  filename, _ = urllib.urlretrieve(url, data_path)
  print("Done !")

In [0]:
# unzip the dataset file. Text has already been processed
with zipfile.ZipFile(data_path) as f:
  text_words = f.read(f.namelist()[0]).lower().split()

In [0]:
# build dictionary and replace rare words with UNK token
count = [('UNK',-1)]

# retrieve most common words
count.extend(collections.Counter(text_words).most_common(max_vocabulary_size-1))

# remove samples with occurences less than 'min_occurences'
for i in range(len(count) -1 , -1, -1):
  if count[i][1] < min_occurence:
    count.pop(i)
    
  else:
    # the collection is ordered, so stop when 'min_occurences' is reached
    break
    
# compute vocabulary size
vocabulary_size = len(count)

In [0]:
# Assign id to each word
word2id = dict()

for i, (word,_) in enumerate(count):
  word2id[word]=i
  
data=list()

unk_count = 0

for word in text_words:
  # retrieve a word id, or assign it an index 0 ('UNK') if not in dictionary
  index = word2id.get(word,0)
  if index == 0:
    unk_count += 1
  
  data.append(index)
  
count[0]=('UNK', unk_count)  
id2word = dict(zip(word2id.values(), word2id.keys()))

In [12]:
print("Word count:", len(text_words))
print("Unique count:", len(set(text_words)))
print("Vocabulary size:", vocabulary_size)
print("Most common words:", count[:10])

Word count: 17005207
Unique count: 253854
Vocabulary size: 47135
Most common words: [('UNK', 444176), ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]


In [0]:
data_index = 0

# Generate training batch for the skip-gram model
def next_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size%num_skips == 0
  assert num_skips <=2*skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size,1), dtype=np.int32)
  
  #get window size (words left and right + current one)
  span = 2*skip_window+1
  buffer = collections.deque(maxlen=span)
  if data_index+span >len(data):
    data_index=0
  
  buffer.extend(data[data_index:data_index+span])
  data_index += span
  
  for i in range(batch_size//num_skips):
    context_words = [w for w in range(span) if w!=skip_window]
    words_to_use = random.sample(context_words, num_skips)
    for j, context_words in enumerate(words_to_use):
      batch[i*num_skips+j]=buffer[skip_window]
      labels[i*num_skips+j, 0]=buffer[context_words]
      
    if data_index ==len(data):
      buffer.extend(data[0:span])
      data_index=span
    else:
      buffer.append(data[data_index])
      data_index +=1
  # backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index+len(data)-span)%len(data)
  return batch, labels

In [0]:
# input data
X = tf.placeholder(tf.int32, shape=[None])
Y = tf.placeholder(tf.int32, shape=[None,1])

In [0]:
# ensure following ops are assigned to cpu
# some are not compatible on gpu
with tf.device('/cpu:0'):
  #create embedding variable (each row represents an embedding variable)
  embedding = tf.Variable(tf.random_normal([vocabulary_size, embedding_size]))
  # lookup corresponding embedding vectors for each sample in x
  x_embed = tf.nn.embedding_lookup(embedding,X)
  
  # construct variables for NCE loss
  nce_weights = tf.Variable(tf.random_normal([vocabulary_size, embedding_size]))
  nce_bias = tf.Variable(tf.zeros([vocabulary_size]))

In [18]:
# compute average nce loss for batch
loss_op = tf.reduce_mean(tf.nn.nce_loss(
weights = nce_weights,
biases = nce_bias,
labels = Y,
inputs = x_embed,
num_sampled = num_sampled,
num_classes = vocabulary_size))

W0815 18:08:05.173580 140421871855488 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/nn_impl.py:180: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss_op)

In [0]:
# Evaluation
# compute the cosine similarity between input data embedding and every embedding vectors
x_embed_norm = x_embed  / tf.sqrt(tf.reduce_sum(tf.square(x_embed)))
embedding_norm = embedding/tf.sqrt(tf.reduce_sum(tf.square(embedding),1,keepdims=True))
cosine_op = tf.matmul(x_embed_norm, embedding_norm, transpose_b=True)

In [0]:
# initializer variables
init = tf.global_variables_initializer()

In [33]:
with tf.Session() as sess:
  sess.run(init)
  
  # testing data
  x_test = np.array([word2id[w] for w in eval_words])
  
  average_loss = 0.0
  
  for step in xrange(1, num_steps+1):
    #get new batch of data
    batch_x, batch_y = next_batch(batch_size, num_skips, skip_window)
    #training op
    _, loss = sess.run([train_op, loss_op], feed_dict ={X:batch_x, Y:batch_y})
    average_loss += loss
    
    if step %display_steps ==0 or step ==1:
      if step >1:
        average_loss /= display_steps
      print("Step", step, "Average loss", average_loss)
      average_loss=0.0
      
    # evaluation
    if step % eval_steps==0 or step ==1:
      print("Evaluation ...")
      sim = sess.run(cosine_op, feed_dict={X:x_test})
      for i in xrange(len(eval_words)):
        top_k = 8 # number of nearest neighbours
        nearest = (-sim[i,:]).argsort()[1:top_k+1]
        log_str = '"%s" nearest neighbour:'%eval_words[i]
        for k in xrange(top_k):
          log_str = '%s %s,'%(log_str, id2word[nearest[k]])
        print(log_str)

Step 1 Average loss 535.07568359375
Evaluation ...
"five" nearest neighbour: gloucester, sulpicius, metamorphic, rectangles, factories, finest, sacrifices, transonic,
"of" nearest neighbour: giordano, wheat, xinjiang, max, insist, nasi, swarm, diagrams,
"going" nearest neighbour: yama, offered, cutaway, stopping, karen, uncertainties, anita, accompanies,
"hardware" nearest neighbour: lise, bring, boa, unisys, steyr, inherit, mathematician, pals,
"american" nearest neighbour: schuschnigg, absolutist, niue, devastates, sterilized, interests, naphtali, cymru,
"britain" nearest neighbour: hearn, compounded, spectroscopy, vulgar, human, seafood, sees, marines,
Step 10000 Average loss 198.35798218307494
Step 20000 Average loss 95.49908768181801
Step 30000 Average loss 64.59485117683411
Step 40000 Average loss 50.02528713760376
Step 50000 Average loss 40.82750349438191
Step 60000 Average loss 34.67690892698765
Step 70000 Average loss 32.04426951887608
Step 80000 Average loss 28.74221265425682